In [2]:
# Import torch
import torch

# Exercises require PyTorch > 1.10.0
print(torch.__version__)

# TODO: Setup device agnostic code


2.0.0


In [3]:
# Check for GPU
!nvidia-smi

Sun Aug 20 16:02:32 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 Ti   WDDM  | 00000000:06:00.0  On |                  N/A |
|  0%   51C    P2              80W / 290W |   2556MiB /  8192MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 1. What are 3 areas in industry where computer vision is currently being used?

1- Autonomous Vehicles

2- Healthcare

3- Retail and E-commerce


### 2. Search "what is overfitting in machine learning" and write down a sentence about what you find

It is to analyze a data too exactly that it may prevent future predictions and therefore unnecessary (that is the first thing that i understood from a 5 minute google search)

In [ ]:
import torchvision 